In [2]:
import pandas as pd
import numpy as np
import requests
from pprint import pprint
from bs4 import BeautifulSoup
import datetime as dt

import re, string, unicodedata
import nltk
import contractions
# from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

# from nltk.stem import LancasterStemmer, WordNetLemmatizer

from api_keys import api_key

In [3]:
def request_song_info(artist_id, page_number):
    base_url = f'https://api.genius.com'
    headers = {f'Authorization': f'Bearer {api_key}'}
    search_url = f'{base_url}/artists/{artist_id}/songs?per_page=40&page={page_number}&sort=popularity'
    response = requests.get(search_url, headers=headers)

    return response

In [4]:
jay_z_dict = {}
beyonce_dict = {}
artist_dict = {2 : jay_z_dict, 498 : beyonce_dict}
        
for artist_id, artist_dict in artist_dict.items():      

    for page in range (1, 20):
        response = request_song_info(artist_id, page)
        json = response.json()
        remote_song_info = None

        for song in json['response']['songs']:
            if str(artist_id) in str(song['primary_artist']['id']):
                remote_song_info = song

                if remote_song_info:
                    song_url = remote_song_info['url']

                if len(artist_dict) == 300:
                    break

                else:
                    artist_dict.update({song['title'].lower() : song_url})

In [5]:
print(f'the jay_z_dict contains {len(jay_z_dict)} songs.')
print(f'the beyonce_dict contains {len(beyonce_dict)} songs.')

the jay_z_dict contains 300 songs.
the beyonce_dict contains 300 songs.


In [6]:
def scrap_song_url(song_url, date_index):
    page = requests.get(song_url)
    html = BeautifulSoup(page.text, 'html.parser')
    release_date = dt.datetime.strptime(html.find_all('span', {'class': 'metadata_unit-info metadata_unit-info--text_only'})[date_index].get_text(), '%B %d, %Y').strftime('%Y%m%d')
    lyrics = html.find('div', class_='lyrics').get_text()

    return release_date, lyrics

In [7]:
relationship_start = scrap_song_url(jay_z_dict["'03 bonnie & clyde"], 1)[0]
print(int(relationship_start))

20021010


In [8]:
pre_jay_z_dict = {}
post_jay_z_dict = {}

for song, song_url in jay_z_dict.items():
    try:
        if int(scrap_song_url(song_url, 1)[0]) >= int(relationship_start):
            if len(post_jay_z_dict) == 50:
                continue
            else:
                post_jay_z_dict.update({song : scrap_song_url(song_url, 1)[1]})
        else:
            if len(pre_jay_z_dict) == 50:
                continue
            else:
                pre_jay_z_dict.update({song : scrap_song_url(song_url, 1)[1]})

    except:
        try:
            if int(scrap_song_url(song_url, 0)[0]) >= int(relationship_start):
                if len(post_jay_z_dict) == 50:
                    continue
                else:
                    post_jay_z_dict.update({song : scrap_song_url(song_url, 0)[1]})
            else:
                if len(pre_jay_z_dict) == 50:
                    continue
                else:
                    pre_jay_z_dict.update({song : scrap_song_url(song_url, 0)[1]})                
                
        except:
            print(f'FAILED on {song}')

FAILED on dead presidents iii
FAILED on history
FAILED on 44 four's
FAILED on a million and one questions (remix)
FAILED on blow the whistle
FAILED on 30 something (remix)
FAILED on open letter part ii (remix)
FAILED on pump it up freestyle
FAILED on if i can't (freestyle)
FAILED on when the money goes
FAILED on addicted to the game
FAILED on 2002 funk flex freestyle
FAILED on dead or alive, pt. 1
FAILED on early this morning


In [9]:
print(len(pre_jay_z_dict))
print(len(post_jay_z_dict))

50
50


In [31]:
pre_beyonce_dict = {}
post_beyonce_dict = {}

for song, song_url in beyonce_dict.items():
    try:
        if int(scrap_song_url(song_url, 1)[0]) >= int(relationship_start):
            if len(post_beyonce_dict) == 50:
                continue
            else:
                post_beyonce_dict.update({song : scrap_song_url(song_url, 1)[1]})
        else:
            if len(pre_beyonce_dict) == 50:
                continue
            else:
                pre_beyonce_dict.update({song : scrap_song_url(song_url, 1)[1]})

    except:
        try:
            if int(scrap_song_url(song_url, 0)[0]) >= int(relationship_start):
                if len(post_beyonce_dict) == 50:
                    continue
                else:
                    post_beyonce_dict.update({song : scrap_song_url(song_url, 0)[1]})
            else:
                if len(pre_beyonce_dict) == 50:
                    continue
                else:
                    pre_beyonce_dict.update({song : scrap_song_url(song_url, 0)[1]})                
                
        except:
            print(f'FAILED on {song}')

FAILED on bow down
FAILED on beyoncé [tracklist]
FAILED on hello
FAILED on roc
FAILED on wake up
FAILED on denial (poem)
FAILED on flaws and all
FAILED on resentment (live at on the run tour)
FAILED on i been on (remix)
FAILED on i go gets out
FAILED on standing on the sun (unfinished version)
FAILED on rise up
FAILED on god made you beautiful
FAILED on still in love (kissing you)
FAILED on beautiful liar (freemasons mix)
FAILED on satellites
FAILED on ghost
FAILED on one night only (disco)
FAILED on if looks could kill (you would be dead)
FAILED on sexy lil' thug (remix)
FAILED on yours and mine
FAILED on apology to fans
FAILED on runway
FAILED on i can't take no more
FAILED on apathy
FAILED on the formation world tour
FAILED on hard to say goodbye
FAILED on what is it gonna be
FAILED on me, myself & i (remix)
FAILED on cards never lie
FAILED on if i were a boy / you oughta know (live at the "52nd grammy awards")
FAILED on fighting temptation
FAILED on slow love
FAILED on letter to mi

In [32]:
print(len(pre_beyonce_dict))
print(len(post_beyonce_dict))

6
50


In [62]:
pre_jay_z_df = pd.DataFrame.from_dict(pre_jay_z_dict, orient = 'index', columns = ['pre_text'])
post_jay_z_df = pd.DataFrame.from_dict(post_jay_z_dict, orient = 'index', columns = ['post_text'])
pre_beyonce_df = pd.DataFrame.from_dict(pre_beyonce_dict, orient = 'index', columns = ['pre_text'])
post_beyonce_df = pd.DataFrame.from_dict(pre_beyonce_dict, orient = 'index', columns = ['post_text'])

In [63]:
stop = stopwords.words('english')

def clean_text(artist_df):
    artist_df = artist_df.apply(lambda x: x.replace('\n', ' '))
    artist_df = artist_df.apply(lambda x: re.sub('\[[^]]*\]', '', x))
    artist_df = artist_df.apply(lambda x: x.lower())
    artist_df = artist_df.apply(lambda x: contractions.fix(x))
    artist_df = artist_df.apply(lambda x: re.sub(r'[^\w\s]','', x))
    artist_df = artist_df.apply(lambda x: [item for item in x.split() if item not in stop])
    return artist_df

In [64]:
pre_jay_z_df.pre_text = clean_text(pre_jay_z_df.pre_text)
post_jay_z_df.post_text = clean_text(post_jay_z_df.post_text)
pre_beyonce_df.pre_text = clean_text(pre_beyonce_df.pre_text)
post_beyonce_df.post_text = clean_text(post_beyonce_df.post_text)

In [65]:
post_beyonce_df.head()

,post_text
work it out,"[honey, baby, know, ask, much, girl, spending,..."
hey goldmember,"[solid, gold, solid, gold, gold, solid, gold, ..."
after all is said and done,"[looking, mirror, open, face, pain, erased, sk..."
love me...,"[yo, let, tell, care, locked, could, smell, pi..."
the spot (jail),"[live, life, fullest, party, every, chance, ge..."


In [ ]:
# pre_jay_z_df.pre_text = pre_jay_z_df.pre_text.apply(lambda x: nltk.word_tokenize(x))

In [56]:
pre_jay_z_df.to_csv('test.csv')